# 12.1 . Simulating screeners
In backtesting, we should be able to simulate what a screener would have given, else we would have to simulate all stocks simultaneously. Because this is a computationally expensive thing to do, the results are saved in the <code>processed/cache/</code> folder. There are of course some constraints to this. It should be technically possible to get the results from screeners, either online or via APIs (e.g. IBKR TWS). This means that intraday scanners are limited. For long-term screeners it does not matter because we can actually download everything after the end of the trading day.

In [1]:
from utils import remove_extended_hours, get_market_dates, get_tickers, get_data, first_trading_date_after_equal
from datetime import datetime, date, timedelta
import mplfinance as mpf
import pandas as pd
import numpy as np
import os
import json
import pyarrow as pa
import pyarrow.parquet as pq
import ast
DATA_PATH = "../data/polygon/"

In [11]:
def store_top_n_liquid(n=500, start = date(2000, 1, 1), end = date(2100, 1, 1)):
    tickers = get_tickers()
    tickers = tickers[tickers['type'] == "CS"]

    dates_and_IDs = {} # {'2022-01-01': ['AAPL', 'MSFT', ...], '2022-04-01': ['NVDA', 'AMD', ...], ...}

    quarterly_all = pd.DataFrame()
    for index, id in enumerate(tickers['ID']):
        bars = get_data(id, columns=['volume', 'close'], start=start, end=end)
        quarterly = bars.resample('Q').agg({'close': 'last',
                                'volume': 'sum'})
        quarterly['turnover'] = quarterly['volume'] * quarterly['close']
        quarterly = quarterly.rename(columns={'turnover': id}).drop(columns=['volume', 'close'])
        quarterly_all = quarterly_all.merge(quarterly[id], how='outer', left_index=True, right_index=True)
        #print(index)
        
        # Avoids defragmentation, increasing performance. Without this it would take more than 4x longer.
        if index % 100 == 0 and index != 0:
            quarterly_all = quarterly_all.copy()
            print(index)

    # Store results in dates_and_IDs
    for datetime_, row in quarterly_all.copy().iterrows():
        top_n_stocks = row[row.notna()].nlargest(n).index.tolist()
        next_trading_date = first_trading_date_after_equal(datetime_.to_pydatetime().date() + timedelta(days=1))
        dates_and_IDs[next_trading_date.isoformat()] = top_n_stocks

    # Store to json (csv is not a convenient format to store variable length lists)
    with open(DATA_PATH + f'processed/cache/top_{n}_liquid.json', 'w') as f: 
        json.dump(dates_and_IDs, f)
    return

def get_top_n_liquid(day, n=500):
    if not os.path.isfile(DATA_PATH + f'processed/cache/top_{n}_liquid.json'):
        store_top_n_liquid(n)
        
    with open(DATA_PATH + f'processed/cache/top_{n}_liquid.json', 'r') as f:
        data = json.load(f)

    dates = list(data.keys())
    date_to_query = max(list(filter(lambda x: x <= day.isoformat(), dates)) )
    return date_to_query, data[date_to_query]

In [13]:
date_, data = get_top_n_liquid(date(2023, 8, 25), n=100)
print(date_)
print(data[:5])

2023-07-03
['TSLA-2019-01-01', 'NVDA-2019-01-01', 'AAPL-2019-01-01', 'MSFT-2019-01-01', 'AMD-2019-01-01']


In [5]:
# Top gainers %

# 2. Testing a mean-reversion strategy on SPY.
I will try out a popular strategy using the IBS indicator. I want to see if I can match the results on [here](https://www.quantifiedstrategies.com/internal-bar-strength-ibs-indicator-strategy/). I will also use pure pandas.

* Universe: SPY ETF
* Entry: IBS < 0.2
* Exit: IBS > 0.8
* Trading on close prices.

Although it is possible to get the exact closing price using market-on-close orders, you cannot know the value of the IBS at market close. So as an extension I will look at how the strategy has performed if the IBS is calculated using daily bars exluding the last minute.

I also want to look at the impact of taxes (36%) and inflation.

Note: I would not use the SPY for trading indices, because futures are more liquid. Even index CFDs may have lower trading costs. Also, I can only access US-domiciled ETFs from a very small amount of brokers due to stupid EU regulations.